### Importar las librerías

In [10]:
import pyodbc

### Crear la Base de Datos

In [11]:
# Colocamos el nombre de nuestro server y dejamos la database master para desde allí crear nuestra base de datos
server = 'DESKTOP-B42O7PT\\SQLEXPRESS'
database = 'master'

# Creamos la conexion y ejecutamos la sentencia que crea nuestra base de datos en la que trabajaremos 
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    # Crear una nueva base de datos
    cursor.execute('CREATE DATABASE NBA_DB')
    print("Base de datos creada exitosamente")
except Exception as e:
    print(f"Error al conectar a SQL Server: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

Base de datos creada exitosamente


### Creamos las tablas en la Base de Datos y las relaciones

In [12]:
# Base de datos en la que crearemos las tablas 
database = 'NBA_DB'

# String conexion
conn_str = (
    'DRIVER={SQL Server};'
    'SERVER=' + server + ';'  
    'DATABASE=' + database + ';' 
    'Trusted_Connection=yes;'
)

# Create a connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Crear un cursor
cursor = conn.cursor()

# Crear tabla Teams
cursor.execute('''
    CREATE TABLE teams (
    team_id INT PRIMARY KEY,
    full_name NVARCHAR(100),
    abbreviation NVARCHAR(100),
    city NVARCHAR(100),
    state NVARCHAR(100),
    arena NVARCHAR(100),
    arenacapacity INT,
    owner NVARCHAR(100),
    generalmanager NVARCHAR(100),
    headcoach NVARCHAR(100),
    year_founded INT,
    year_active_till INT,
    )
''')

# Crear tabla Games
cursor.execute('''
    CREATE TABLE games (
    game_id INT PRIMARY KEY,
    season_id INT,
    team_id_home INT,
    team_id_away INT,
    game_date NVARCHAR(100),
    wl_home NVARCHAR(100),
    wl_away NVARCHAR(100),
    pts_home INT,
    pts_away INT,
    attendance INT,
    game_time NVARCHAR(100),
    season INT,
    team_city_name_home NVARCHAR(100),
    team_nickname_home NVARCHAR(100),
    team_city_name_away NVARCHAR(100),
    team_nickname_away NVARCHAR(100),
    FOREIGN KEY (team_id_home) REFERENCES teams(team_id),
    FOREIGN KEY (team_id_away) REFERENCES teams(team_id)
    )
''')

# Crear tabla Drafts
cursor.execute('''
    CREATE TABLE drafts (
    player_id INT PRIMARY KEY,
    weight FLOAT,
    wingspan FLOAT,
    standing_reach FLOAT,
    body_fat_pct FLOAT,
    standing_vertical_leap FLOAT,
    max_vertical_leap FLOAT,
    lane_agility_time FLOAT,
    modified_lane_agility_time FLOAT,
    three_quarter_sprint FLOAT,
    bench_press INT,
    player_name NVARCHAR(100),
    season INT,
    round_number INT,
    round_pick INT,
    overall_pick INT,
    team_id INT,
    organization NVARCHAR(100),
    organization_type NVARCHAR(100),
    )
''')

# Crear tabla Players
cursor.execute('''
    CREATE TABLE players (
    player_id INT PRIMARY KEY,
    height NVARCHAR(100),
    weight INT,
    season_exp INT,
    jersey NVARCHAR(100),
    position NVARCHAR(100),
    team_id INT,
    team_name NVARCHAR(100),
    team_city NVARCHAR(100),
    full_name NVARCHAR(100),
    is_active INT,
    )
''')

# Crear tabla Other Stats
cursor.execute('''
    CREATE TABLE other_stats (
    game_id INT PRIMARY KEY,
    team_id_home INT,
    team_city_home NVARCHAR(100),
    pts_paint_home INT,
    times_tied INT,
    total_turnovers_home INT,
    team_id_away INT,
    team_city_away NVARCHAR(100),
    pts_paint_away INT,
    total_turnovers_away INT,
    )
''')

# Crear tabla NBA Salaries
cursor.execute('''
    CREATE TABLE nba_salaries (
    full_name NVARCHAR(100),
    Salary INT,
    Position NVARCHAR(100),
    Age INT,
    Team NVARCHAR(100),
    PTS FLOAT,
    player_id INT PRIMARY KEY,
    is_active INT,
    )
''')

### Aquí se deben cargar los datos a la DB antes de crear las relaciones

In [13]:
#----------------------- SE CREAN LAS RELACIONES -----------------------#

cursor.execute('''
ALTER TABLE drafts
ADD CONSTRAINT FK_drafts_players
FOREIGN KEY(player_id) REFERENCES players(player_id);
''')

cursor.execute('''
ALTER TABLE players
ADD CONSTRAINT FK_players_teams
FOREIGN KEY(team_id) REFERENCES teams(team_id);
''')

cursor.execute('''
ALTER TABLE other_stats
ADD CONSTRAINT FK_other_stats_games
FOREIGN KEY(game_id) REFERENCES games(game_id);
''')

cursor.execute('''
ALTER TABLE nba_salaries
ADD CONSTRAINT FK_nba_salaries_players
FOREIGN KEY(player_id) REFERENCES players(player_id);
''')

# Confirmar los cambios
conn.commit()

# Cerrar la conexión
cursor.close()
conn.close()